<a href="https://colab.research.google.com/github/dev-tyta/dogs_vs_cats/blob/main/dogs_cats_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! pip install google

     -------------------------------------- 59.0/59.0 kB 312.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 78.2/78.2 kB 127.9 kB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73057 sha256=ce111f3abdccbd58a6e3d516ad9e78605ef3720d8774fc4918cc5091fccc3d9c
  Stored in directory: c:\users\testys\appdata\local\pip\cache\wheels\29\da\11\144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggleCollecting kaggle
     -------------------------------------- 59.0/59.0 kB 103.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'

     -------------------------------------- 78.2/78.2 kB 124.2 kB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73057 sha256=ce111f3abdccbd58a6e3d516ad9e78605ef3720d8774fc4918cc5091fccc3d9c


The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


     --------------------------------------- 45.3/45.3 kB 79.9 kB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd /content/drive/MyDrive/dogs_cats

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/dogs_cats"

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!unzip dogs-vs-cats.zip

In [ ]:
!unzip train.zip
!unzip test1.zip

In [ ]:
import tensorflow as tf
import os
from pathlib import Path
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Sequential, layers

In [ ]:
train_dir= "train"
test_dir= "test1"

In [ ]:
train_df = pd.DataFrame({'image_name':os.listdir(train_dir)})
train_df['label'] =train_df['image_name'].apply(lambda x: x.split('.')[0])
train_df

In [ ]:
test_df = pd.DataFrame({'image_name':os.listdir(test_dir)})
test_df['label'] =test_df['image_name'].apply(lambda x: x.split('.')[0])
test_df

In [ ]:
dog_path_train = os.path.join(train_dir, 'dog')
os.mkdir(dog_path_train)
dog_df_train = train_df[train_df.label=='dog']


In [ ]:
for n in tqdm(dog_df_train.image_name):
    os.rename((os.path.join(train_dir, n)), (os.path.join(dog_path_train, n)))

In [ ]:
cat_path_train = os.path.join(train_dir, 'cat')
os.mkdir(cat_path_train)
cat_df_train = train_df[train_df.label=='cat']


In [ ]:
for n in tqdm(cat_df_train.image_name):
    os.rename((os.path.join(train_dir, n)), (os.path.join(cat_path_train, n)))

In [ ]:
train_datagen = ImageDataGenerator(rescale= 1./255, validation_split= 0.3)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    class_mode='binary',
    subset='training'
)

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    class_mode= 'binary',
    subset= 'validation'
)

In [ ]:
model = Sequential([
    layers.Conv2D(16, (3,3), activation= 'relu', input_shape =(150, 150, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation= 'relu'),
    layers.Dense(1,activation= 'sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss= 'binary_crossentropy',
             optimizer = RMSprop(learning_rate= 0.001),
              metrics= ['accuracy']
             )

In [ ]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if logs.get('val_accuracy')>=0.90:
            self.model.stop_training = True
            
callback = mycallback()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//50,#batch_size, 
    epochs = 30,
    verbose=1,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//50,#batch_size,
    callbacks=[callback]
)

In [ ]:
model.save("dogs_cats.h5")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

#new = Path.('\dogs-vs-cats\test1')

images = os.listdir(r'test1')
print(images)



In [ ]:
for i in images:
    print()
    # predicting images
    path = r'test1/' + i
    # print(path)
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    results = []
    if classes[0]>0.5:
        results.append('Dog')
    if classes[0]< 0.5:
        results.append('Cat')

In [ ]:
print(results)

In [ ]:
img = os.listdir(r'C:\Users\Hp 15\Documents\My_files\Image Classification\tmp')
print(img)

In [ ]:
for i in images:
    print()
    # predicting images
    path = r'C:\Users\Hp 15\Documents\My_files\Image Classification\tmp/' + i
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    results = []
    if classes[0]>0.5:
        results.append('Dog')
    else:
        results.append('Cat')

In [ ]:
print(results)